## 10-714: Homework 0

The goal of this homework is to give you a quick overview of some of the concepts and ideas that you should be familiar with _prior_ to taking this course.  The assignment will require you to build a basic softmax regression algorithm, plus a simple two-layer neural network.  You will create these implementations both in native Python (using the numpy library), and (for softmax regression) in native C/C++.  The homework will also walk you through the process of submitting your assignments to our autograding system.  Along the way, we'll give some guidance as to how you might want to implement these different functions, but overall the details are up to you.  What we will say, though, is that in the Python version you should make heavy use of the linear algebra calls in numpy: trying to use explicit loops will usually make the code much slower than it should be.

**We know that there is a lot of prose text in this assignment, especially in the beginning, and relatively little coding.  That being said, _please_ read carefuly through the entirety of the text in this writeup.  Doing so will describe the process and philosophy behind how we structure our assignments, and will make a huge difference in your ability to complete later assignments.**

All the code development for the homeworks in 10-714 can be done in the Google Colab environment.  However, instead of making extensive use of actual code blocks within a colab notebook, most of the code you develop will be done `.py` files downloaded (automatically) to your Google Drive, and you will largely use the notebook for running shell scripts that test and submit the code to the auto-grader (or optionally for testing out snippets of code as you develop, but this is not required).  This is a somewhat non-standard usage of Colab Notebooks (typically one uses them more like interactive coding environments, with code cells directly in the notebook).  However, our rationale for using them in the way that we do is actually straightforward: in addition to being a nice cloud-based notebook environment, Colab also provides very nice access to "standard" cloud-based GPU systems that you can spin up quickly, which will let you develop especially some of the later (CUDA-based) code without requiring you to get access to a physical GPU, or set up the CUDA libraries yourself.  That being said, **you are welcome to do the development and submission of your code in any environment you like**, we just can't guarantee the abiltiy to support any environment other than the Colab-based one.

To get started, **make a copy of this notebook** file by selecting "Save a copy in Drive" from the "File" menu, and then run the code block below.  This will load your Google Drive folder into the Colab notebook environment, create a `/10714/hw0` directory, and clone the HW0 public repository into this directory.

## 10-714: 作业 0

本作业的目的是让您快速了解在学习本课程之前应该熟悉的一些概念和观点。 作业要求您构建一个基本的软最大回归算法和一个简单的双层神经网络。 您将在本地 Python（使用 numpy 库）和本地 C/C++ 中创建这些实现（对于 softmax 回归）。 作业还将指导您如何向我们的自动交易系统提交作业。 一路上，我们会就如何实现这些不同的功能提供一些指导，但总体而言，细节取决于您自己。 不过，我们要说的是，在 Python 版本中，你应该大量使用 numpy 中的线性代数调用：试图使用显式循环通常会使代码变得比应有的速度慢得多。

**我们知道，本作业中有大量的散文，尤其是在开头，而编码相对较少。 尽管如此，请仔细阅读本文的全部内容。 这样做将描述我们如何组织作业的过程和背后的理念，并将对你完成后面作业的能力产生巨大的影响。

10-714 中所有作业的代码开发都可以在 Google Colab 环境中完成。 但是，您开发的大部分代码都将以`.py`文件的形式下载（自动）到您的谷歌硬盘中，而不是在 Colab 笔记本中大量使用实际代码块，您将主要使用笔记本来运行 shell 脚本，以测试并向自动评分器提交代码（也可以在开发过程中测试代码片段，但这不是必需的）。 这有点不符合 Colab 笔记本的使用标准（通常情况下，使用 Colab 笔记本更像是交互式编码环境，直接在笔记本中使用代码单元）。 不过，我们使用这种方式的理由其实很简单：Colab除了是一个不错的基于云的笔记本环境外，还提供了很好的访问 "标准 "云GPU系统的途径，您可以快速启动这些系统，这样您就可以开发特别是一些后期（基于CUDA）的代码，而无需访问物理GPU或自己设置CUDA库。 尽管如此，**欢迎您在任何环境中开发和提交代码**，我们只是无法保证支持除基于 Colab 的环境之外的任何环境。

要开始使用，**从 "文件 "菜单中选择 "在驱动器中保存副本"，复制本笔记本**文件，然后运行下面的代码块。 这将把你的 Google Drive 文件夹加载到 Colab 笔记本环境中，创建一个 `/10714/hw0` 目录，并将 HW0 公共仓库克隆到该目录中。

In [ ]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir -p 10714
%cd /content/drive/MyDrive/10714
!git clone https://github.com/dlsyscourse/hw0.git
%cd /content/drive/MyDrive/10714/hw0

This next cell will then install the libraries required.

下一个单元将安装所需的程序库。

In [ ]:
!pip3 install --upgrade --no-deps git+https://github.com/dlsyscourse/mugrade.git
!pip3 install pybind11
!pip3 install numdifftools

## Question 1: A basic `add` function, and testing/autograding basics

To illustrate the workflow of these assignments and the autograding system, we'll use a simple example of implementing an `add` function.  Note that the commands run above will create the following structure in your `10714/hw0` directory

    data/
        train-images-idx3-ubyte.gz
        train-labels-idx1-ubyte.gz
        t10k-images-idx3-ubyte.gz
        t10k-labels-idx1-ubyte.gz
    src/
        simple_ml.py
        simple_ml_ext.cpp
    tests/
        test_simple_ml.py
    Makefile
    
The `data/` directory contains the data needed for this assignment (a copy of the MNIST data set); the `src/` directory contains the source files where you will write your implementations; the `tests/` directory contains tests that will evaluate (locally) your solution, and also submit them for autograding.  And the `Makefile` file is a makefile that will compile the code (relevant for the C++ portions of the assignment).

The first homework question requires you to implement `simple_ml.add()` function (this trivial function is not used anywhere, it is just an example to get you used to the structure of the assignment).  Looking at the `src/simple_ml.py` file, you will find the following function stub for the `add()` function.

```python
def add(x, y):
    """ A trivial 'add' function you should implement to get used to the 
    autograder and submission system.  The solution to this problem is in
    the homework notebook.

    Args:
        x (Python number or numpy array)
        y (Python number or numpy array)

    Return:
        Sum of x + y
    """
    ### YOUR CODE HERE
    pass
    ### END YOUR CODE
```
The docstring in each file defines the expected input/output mapping that your function should produce (you need to get used to reading carefully, as the number one source of errors in submissions, we typically find,  is simply not reading the spec).  And hopefully it's pretty obvious to you how to implement this function.  You would just replace the `pass` statement with the correct code, namely the following:

```python
def add(x, y):
    """ A trivial 'add' function you should implement to get used to the 
    autograder and submission system.  The solution to this problem is in the
    the homework notebook.

    Args:
        x (Python number or numpy array)
        y (Python number or numpy array)

    Return:
        Sum of x + y
    """
    ### YOUR CODE HERE
    return x + y
    ### END YOUR CODE
```
Go ahead and do this in your `src/simple_ml.py` file.

## 问题 1：基本的 `add` 函数和测试/自动交易基础知识

为了说明这些作业和自动交易系统的工作流程，我们将使用一个简单的例子来实现一个 `add` 函数。 请注意，上面运行的命令将在您的 `10714/hw0` 目录中创建以下结构

    data/
        train-images-idx3-ubyte.gz
        train-labels-idx1-ubyte.gz
        t10k-images-idx3-ubyte.gz
        t10k-labels-idx1-ubyte.gz
    src/
        simple_ml.py
        simple_ml_ext.cpp
    tests/
        test_simple_ml.py
    Makefile

data/"目录包含本作业所需的数据（MNIST 数据集的副本）；"src/"目录包含源文件，您将在其中编写实现方法；"tests/"目录包含测试，这些测试将评估（本地）您的解决方案，并将其提交给自动升级。 而 `Makefile` 文件则是编译代码的 makefile（与作业的 C++ 部分相关）。

第一道作业题要求你实现 `simple_ml.add()` 函数（这个琐碎的函数在任何地方都用不到，它只是一个让你熟悉作业结构的例子）。 查看 `src/simple_ml.py` 文件，你会发现下面的 `add()` 函数存根。

```python
def add(x, y):
    """ A trivial 'add' function you should implement to get used to the 
    autograder and submission system.  The solution to this problem is in
    the homework notebook.

    Args:
        x (Python number or numpy array)
        y (Python number or numpy array)

    Return:
        Sum of x + y
    """
    ### YOUR CODE HERE
    pass
    ### END YOUR CODE
```

每个文件中的 docstring 定义了您的函数应产生的预期输入/输出映射（您需要养成仔细阅读的习惯，因为我们通常会发现，在提交过程中出现错误的首要原因就是没有阅读规范）。 希望你能明白如何实现这个函数。 你只需将 `pass` 语句替换为正确的代码，即以下代码：


```python
def add(x, y):
    """ A trivial 'add' function you should implement to get used to the 
    autograder and submission system.  The solution to this problem is in the
    the homework notebook.

    Args:
        x (Python number or numpy array)
        y (Python number or numpy array)

    Return:
        Sum of x + y
    """
    ### YOUR CODE HERE
    return x + y
    ### END YOUR CODE
```

在 `src/simple_ml.py` 文件中，请完成上述修改。


### Running local tests

Now you will want to test to see if your code works, and if so, to submit it to the autograding system.  Throughout this course, we are using standard tools for running unit tests on code, namely the `pytest` system.  Once you've written the correct code in the `src/simple_ml.py` file, run the following command below.

#### 运行本地测试

现在，您需要测试代码是否有效，如果有效，则将其提交给自动交易系统。 在整个课程中，我们将使用标准工具，即 `pytest` 系统，对代码运行单元测试。 在 `src/simple_ml.py` 文件中写入正确的代码后，运行下面的命令。

In [23]:
!python3 -m pytest -k "add"

============================= test session starts ==============================
platform darwin -- Python 3.7.3, pytest-4.3.1, py-1.8.0, pluggy-0.9.0
rootdir: /Users/zkolter/Dropbox/class/10-714/homework/hw0, inifile:
plugins: remotedata-0.3.1, openfiles-0.3.2, doctestplus-0.3.0, arraydiff-0.3
collected 6 items / 5 deselected / 1 selected                                  

tests/test_simple_ml.py .                                                [100%]

==================== 1 passed, 5 deselected in 0.26 seconds ====================


If all goes correctly, you will see that one tests is passed correctly.  To see how this test works, take a look at the `tests/test_simple_ml.py` file, specifically the `test_add()` function:

```python
def test_add():
    assert add(5,6) == 11
    assert add(3.2,1.0) == 4.2
    assert type(add(4., 4)) == float
    np.testing.assert_allclose(add(np.array([1,2]), np.array([3,4])),
                               np.array([4,6]))
```

This code runs a suite of unit tests against your implemented function.  If the function is implemented correctly, then all the assertions above _should_ pass (i.e., the code will execute without errors).  If on the other hand, you implemented something incorrectly (say, changed the `x + y` above to `x - y`), then these assertions will fail, and `pytest` will indicate that the corresponding test failed.

如果一切顺利，您将看到一个测试正确通过。 要了解该测试如何工作，请查看 `tests/test_simple_ml.py` 文件，特别是 `test_add()` 函数：

```python
def test_add()：
    assert add(5,6) == 11
    assert add(3.2,1.0) == 4.2
    assert type(add(4., 4)) == float
    np.testing.assert_allclose(add(np.array([1,2]), np.array([3,4]))、
                               np.array([4,6]))
```

这段代码针对您执行的函数运行一套单元测试。 如果函数的实现是正确的，那么上述所有断言_应该_通过（即代码执行时不会出错）。 反之，如果您的实现不正确（例如，将上面的 `x + y` 改为 `x - y`），那么这些断言就会失败，`pytest` 也会指出相应的测试失败。

In [24]:
# in this example cell, we replaced "x + y" with "x - y" in simple_ml.add()
!python3 -m pytest -k "add"

============================= test session starts ==============================
platform darwin -- Python 3.7.3, pytest-4.3.1, py-1.8.0, pluggy-0.9.0
rootdir: /Users/zkolter/Dropbox/class/10-714/homework/hw0, inifile:
plugins: remotedata-0.3.1, openfiles-0.3.2, doctestplus-0.3.0, arraydiff-0.3
collected 6 items / 5 deselected / 1 selected                                  

tests/test_simple_ml.py F                                                [100%]

=================================== FAILURES ===================================
___________________________________ test_add ___________________________________

    def test_add():
>       assert add(5,6) == 11
E       assert -1 == 11
E        +  where -1 = add(5, 6)

tests/test_simple_ml.py:16: AssertionError
==================== 1 failed, 5 deselected in 0.35 seconds ====================


As you can see, you will get an error that indicates the line where the assertion failed, which you can then use to go back and debug your implementation.  **You should get comfortabale with reading and tracing through the tests file as a way of better understanding how your implementations should work.**

Learning to properly develop and use unit tests is crucial to modern software development, and hopefully a secondary outcome of this course is that you become familiar with the typical usage of unit tests within software development.  Of course, this isn't entirely true, because you don't necessarily need to _write_ your own tests to pass the questions here, but you _should_ become familiar with how to read the test files that we provide, as a way to understand how your function should behave.  However, we _absolutely_ would also encourage you to write additional tests for your implementations, especially if you find that your code is passing the local tests, but still seems to be failing on submission.

One last quick comment.  If you're used to debugging code via print statements, note that **pytest will by default capture any output**. You can disable this behavior and have the tests display all output in all cases by passing the `-s` flag to pytest.

正如你所看到的，你将会得到一个错误信息，指出断言失败的行，然后你可以利用这个错误信息回头调试你的实现。 **你应该熟悉阅读和跟踪测试文件，以便更好地理解你的实现应该如何工作。

学习正确开发和使用单元测试对现代软件开发至关重要，希望本课程的第二项成果是让你熟悉单元测试在软件开发中的典型用法。 当然，这并不完全正确，因为你并不一定需要编写自己的测试才能通过这里的问题，但你应该熟悉如何阅读我们提供的测试文件，以此了解你的函数应该如何运行。 不过，我们也鼓励您为自己的实现编写额外的测试，尤其是当您发现自己的代码通过了本地测试，但在提交时似乎仍然失败时。

最后再简单说一句。 如果你习惯于通过打印语句调试代码，请注意**pytest 默认会捕获任何输出**。通过向 pytest 传递 `-s` 标志，你可以禁用这一行为，让测试在任何情况下都显示所有输出。

### Submitting to the autograder

Now that you have passed the unit tests, it is time to submit your solution for autograding.  For autograding in this class we are using a custom application written by the course instructors.  To start the autograding progress, go to https://mugrade.org/courses/1?enroll=664EqzbhsV5p99rbBYf0 and login in **using your andrew email** through the Google sign in.

Once you've created an account, click Deep Learning Systems to enter the course. Near the top-right of the page, click **Show API Key** and copy the associated key.  This key is linked to _your submissions_ for the course, and anyone with the key could submit your assignments; therefore, you should not share this key with anyone, and just use it to submit your own code below.  Once you have this key, run the following command.

### 提交给自动交易系统

现在，您已经通过了单元测试，是时候提交您的解决方案进行自动交易了。 在本课中，我们将使用由课程讲师编写的自定义应用程序进行自动交易。 要开始自动交易，请访问 https://mugrade.org/courses/1?enroll=664EqzbhsV5p99rbBYf0，并通过谷歌登录**使用你的安德鲁邮箱**登录。

创建账户后，点击 "深度学习系统 "进入课程。在页面右上角附近，点击**显示 API 密钥**并复制相关密钥。 此密钥与你提交的课程相关联，任何拥有此密钥的人都可以提交你的作业；因此，你不应该与任何人共享此密钥，只需用它在下面提交你自己的代码即可。 拿到密钥后，运行以下命令。

In [ ]:
!python3 -m mugrade submit YOUR_GRADER_KEY_HERE hw0 -k "add"

Running this command will submit your `add` function to the mugrade autograding system.  To see how this works internally, take a look at the `tests/test_simply_ml.py` file again, but this time the `submit_add()` function right below the `test_add` function.

```python
def submit_add():
    mugrade.submit(add(1,2))
    mugrade.submit(add(4.5, 3.2))
    mugrade.submit(type(add(3,2)))
    mugrade.submit(add(np.array([1.,2.]), np.array[5,6]))
```

This code looks a bit like a unit test above, but instead of assertions there are calls to `mugrade.submit()`.  These calls each evaluate the `add` function on different inputs, then send the result to the mugrade server.  The server compares the output of your function with the correct output (which is only stored on the server, not locally, so you cannot know in advance what the correct answer is), and updates your score for the assignment accordingly.  If you are logged into the mugrade system, you can go to the "Homework 0" assignment to see your updated grade (refresh the page as necessary if you are already viewing the page).

**Important note:** For those of you familiar with autograding systems, you might notice that mugrade works a bit differently than most.  In most autograding systems you write your code to pass local tests (if you're lucky, some classes don't even provide these), then bundle up your code and submit it to the autograding system, which will unpack and execute your code on the autograding server, running it against some (unknown, for you) test cases.  Mugrade is different: the `submit_add` function is run on _your system_ (e.g., in the Colab environment you are running), and only the _results_ of the calls are sent to the server.

The rationale for this setup is a subtle but important one.  This class requires you to develop a fairly complex system, which runs your code on specialized hardware like GPUs, on potentially long tests that train real neural network architectures.  Being unable to debug the execution of your code on the autograding test cases poses a substantial challenge in practice, not to mention issues of server capacity and slowdowns that occur usually right at the grading deadline.  Moving computation locally means that you have a lot more insight into how your code is actually running on the autograder test cases, which is extremely valuable for debugging.  By _far_ the most common post on forums for similar courses is "my code passes all the local tests but fails on the autograder"; and while this situation can _still_ happen in mugrade, you can at least step through the execution of your code to see _where_ and _how_ it fails.  And because the server only needs to perform simple checks of your input against the correct outputs, the server is very simple, and you will get the feedback from the autograder immediately, even right at the submission deadline.

The _disadvantage_ of a grading system like this is that it's possible to cheat.  Because you have complete control over the local execution of your code on the autograding test cases, you could, in theory, just figure out what the correct answers should be, and return them without actually implementing the required code.  Later in this assignment, for instance, you're required to write a C++ implementation of softmax regression in addition to the initial Python one.  It would be completely possible to change the autograder to use your Python implementation, rather than the C++ one, and still pass the tests.  That being said, **please do not attempt to circumvent the autograding system in any way**.  Our goal for this system is to make it genuinely easier for your to debug and develop your code, and to make the experience more similar to the "real" development process, and less a matter of "figuring out why the autograder doesn't compile your particular variant of CUDA code".  To address this point, in addition to submitting the autograder tests, when you are finished with your submission you need to also upload a `.tar.gz` file of your `/src` directory in your solution to the mugrade system.  We trust that this will not be an issue, but if there _are_ any concerns, the TAs will always be able to validate that your code produces the results (they will also be running against standard plagiarism detection systems like MOSS).

为了解决这个问题，除了提交 autograder 测试之外，您还需要在完成提交后将解决方案中 `/src` 目录下的 `.tar.gz` 文件上传到 mugrade 系统。 我们相信这不会成为一个问题，但如果有任何顾虑，助教将始终能够验证您的代码是否产生了结果（他们还将运行标准的剽窃检测系统，如 MOSS）。运行此命令将向 mugrade 自动交易系统提交您的 `add` 函数。 要了解内部是如何工作的，请再次查看 `tests/test_simply_ml.py` 文件，但这次`submit_add()`函数就在`test_add`函数下面。

```python

def submit_add()：
    mugrade.submit(add(1,2))
    mugrade.submit(add(4.5, 3.2))
    mugrade.submit(type(add(3,2))
    mugrade.submit(add(np.array([1.,2.]), np.array[5,6]))
```

这段代码看起来有点像上面的单元测试，但调用的不是断言，而是 `mugrade.submit()`。 这些调用分别对不同输入的 `add` 函数进行评估，然后将结果发送到 mugrade 服务器。 服务器会将你的函数输出与正确的输出进行比较（正确的输出只存储在服务器上，而不是本地，因此你无法事先知道正确答案是什么），并相应地更新你的作业得分。 如果您已登录 mugrade 系统，则可进入 "家庭作业 0 "作业查看更新后的成绩（如果您已在查看该页面，则可根据需要刷新页面）。

**重要提示：** 熟悉自动交易系统的人可能会注意到，mugrade 的工作方式与大多数系统有些不同。 在大多数自动交易系统中，您编写的代码都能通过本地测试（如果幸运的话，有些类甚至不提供本地测试），然后将代码捆绑起来并提交给自动交易系统，自动交易系统将解压缩并在自动交易服务器上执行您的代码，并根据某些（对您来说未知的）测试用例运行代码。 Mugrade 则不同："submit_add "函数在_你的系统_（例如，在你运行的 Colab 环境中）上运行，只有调用的_结果_才会发送到服务器。

这种设置的原理很微妙，但也很重要。 这门课要求你开发一个相当复杂的系统，在 GPU 等专用硬件上运行你的代码，进行可能很长时间的测试，训练真实的神经网络架构。 在自动交易测试用例中无法调试代码的执行，这在实践中是一个巨大的挑战，更不用说服务器容量和速度减慢等问题了，而这些问题通常会在评分截止日期前发生。 将计算转移到本地意味着您可以更深入地了解您的代码在自动交易测试用例上的实际运行情况，这对于调试非常有价值。 在类似课程的论坛上，最常见的帖子是 "我的代码通过了所有本地测试，但在 autograder 上却失败了"；虽然这种情况在 mugrade 中仍有可能发生，但您至少可以逐步查看代码的执行情况，以了解失败的位置和原因。 而且，由于服务器只需根据正确的输出对您的输入进行简单的检查，因此服务器非常简单，您甚至可以在提交截止日期前立即从自动交易程序中获得反馈。

这样的评分系统的缺点是有可能作弊。 因为你可以完全控制自动评分测试用例中代码的本地执行，所以理论上，你可以只计算出正确答案，然后返回正确答案，而不实际执行所需的代码。 例如，在本作业的稍后部分，除了最初的 Python 实现外，你还需要编写 softmax 回归的 C++ 实现。 你完全可以把自动跟踪器改成使用你的 Python 实现，而不是 C++ 实现，这样仍然可以通过测试。 尽管如此，**请不要试图以任何方式规避自动交易系统**。 我们开发该系统的目的是让您能真正更轻松地调试和开发您的代码，让您的体验更接近 "真实 "的开发过程，而不是 "弄清为什么自动交易系统无法编译您的特定 CUDA 代码"。 

## Question 2: Loading MNIST data

Now that you're familiar with the autograding system, try it out on the next function you need to implement in the `src/simple_ml.py` file: the `parse_mnist_data()` function.  Here is the function declaration from the file (we typically won't walk through this whole process again, but will do so here one more time).

```python
def parse_mnist(image_filename, label_filename):
    """ Read an images and labels file in MNIST format.  See this page:
    http://yann.lecun.com/exdb/mnist/ for a description of the file format.

    Args:
        image_filename (str): name of gzipped images file in MNIST format
        label_filename (str): name of gzipped labels file in MNIST format

    Returns:
        Tuple (X,y):
            X (numpy.ndarray[np.float32]): 2D numpy array containing the loaded 
                data.  The dimensionality of the data should be 
                (num_examples x input_dim) where 'input_dim' is the full 
                dimension of the data, e.g., since MNIST images are 28x28, it 
                will be 784.  Values should be of type np.float32, and the data 
                should be normalized to have a minimum value of 0.0 and a 
                maximum value of 1.0 (i.e., scale original values of 0 to 0.0 
                and 255 to 1.0).

            y (numpy.ndarray[dtype=np.uint8]): 1D numpy array containing the
                labels of the examples.  Values should be of type np.uint8 and
                for MNIST will contain the values 0-9.
    """
    ### BEGIN YOUR CODE
    pass
    ### END YOUR CODE
```

Hopefully you're now familiar with how this docstring works, and have an idea about how to go about implementing this function.  First, go to http://yann.lecun.com/exdb/mnist/ or this alternate [link](https://web.archive.org/web/20220509025752/http://yann.lecun.com/exdb/mnist/) (the bottom of the page) to read about the binary format for the MNIST data.  Then write a loader that will read files of this type, and return numpy arrays according to the specification in the docstring (if you're having any issues with the implementation, be sure to read the docstring closely).  We'd recommend you use the `struct` module in python (along with the `gzip` module and of course `numpy` itself), in order to implement this function.

When you've implemented the function run the local unit tests.

## 问题 2：加载 MNIST 数据

现在您已经熟悉了自动交易系统，请在 `src/simple_ml.py` 文件中执行下一个函数：`parse_mnist_data()` 函数。 下面是该文件中的函数声明（我们通常不会再走一遍整个过程，但这里会再走一遍）。

```python
def parse_mnist(image_filename, label_filename)：
    """ 读取 MNIST 格式的图像和标签文件。 请参见此页：
    http://yann.lecun.com/exdb/mnist/ 获取文件格式说明。

    参数：
        image_filename (str)：以 MNIST 格式压缩的图像文件的名称
        label_filename (str)：以 MNIST 格式压缩的标签文件名称

    返回值
        元组 (X,y)：
            X（numpy.ndarray[np.float32]）：包含加载数据的 2D numpy 数组。 
                数据的二维 numpy 数组。 数据的维度应为 
                (num_examples x input_dim)，其中 "input_dim "是数据的全维度，例如 
                例如，由于 MNIST 图像的尺寸是 28x28，所以它 
                为 784。 值的类型应为 np.float32，数据 
                的最小值为 0.0，最大值为 1.0。 
                最大值为 1.0（即，将原始值 0 缩放为 0.0，将 255 缩放为 1.0 
                和 255 分辨率为 1.0）。

            y（numpy.ndarray[dtype=np.uint8]）： 1D numpy 数组，包含
                数组。 值的类型应为 np.uint8，且
                对于 MNIST 将包含 0-9 的值。
    """
    #### 开始代码
    pass
    ### 结束代码
```

希望你现在已经熟悉了这个文档字符串是如何工作的，并知道如何去实现这个函数。 首先，访问 http://yann.lecun.com/exdb/mnist/ 或这个备用 [链接](https://web.archive.org/web/20220509025752/http://yann.lecun.com/exdb/mnist/)（页面底部），阅读有关 MNIST 数据二进制格式的信息。 然后编写一个加载器，读取这种类型的文件，并根据 docstring 中的规范返回 numpy 数组（如果在实现过程中遇到任何问题，请务必仔细阅读 docstring）。 我们建议你使用 python 中的 `struct` 模块（以及 `gzip` 模块，当然还有 `numpy` 本身）来实现这个函数。

实现函数后，运行本地单元测试。

In [ ]:
!python3 -m pytest -k "parse_mnist"

And then submit your code to mugrade.

然后将代码提交给 mugrade。

In [ ]:
!python3 -m mugrade submit YOUR_GRADER_KEY_HERE hw0 -k "parse_mnist"

## Question 3: Softmax loss

Implement the softmax (a.k.a. cross-entropy) loss as defined in `softmax_loss()` function in `src/simple_ml.py`.  Recall (hopefully this is review, but we'll also cover it in lecture on 9/1), that for a multi-class output that can take on values $y \in \{1,\ldots,k\}$, the softmax loss takes as input a vector of logits $z \in \mathbb{R}^k$, the true class $y \in \{1,\ldots,k\}$ returns a loss defined by
\begin{equation}
\ell_{\mathrm{softmax}}(z, y) = \log\sum_{i=1}^k \exp z_i - z_y.
\end{equation}

Note that as described in its docstring, `softmax_loss()` takes a _2D array_ of logits (i.e., the $k$ dimensional logits for a batch of different samples), plus a corresponding 1D array of true labels, and should output the _average_ softmax loss over the entire batch.  Note that to do this correctly, you should _not_ use any loops, but do all the computation natively with numpy vectorized operations (to set expectations here, we should note for instance that our reference solution consists of a single line of code).

Note that for "real" implementation of softmax loss you would want to scale the logits to prevent numerical overflow, but we won't worry about that here (the rest of the assignment will work fine even if you don't worry about this).  The code below runs the test cases.

## 问题 3：Softmax loss

实现 `src/simple_ml.py` 中的 `softmax_loss()` 函数中定义的 softmax（又称交叉熵）损失。 回想一下（希望这是复习，但我们也会在 9/1 的讲座中讲到），对于可以取值 $y \in \{1,\ldots,k\}$ 的多类输出，softmax loss 将 logits 向量 $z \in \mathbb{R}^k$作为输入，真类 $y \in \{1,\ldots,k\}$ 返回的损失定义为
\开始
\ell_{\mathrm{softmax}}(z, y) = \log\sum_{i=1}^k \exp z_i - z_y。
\end{equation}

请注意，正如文档中所述，"softmax_loss() "需要一个对数的_2D 数组（即一批不同样本的 $k$ 维对数），加上一个相应的真实标签的 1D 数组，并输出整批样本的_平均_软最大损失。 需要注意的是，要正确地实现这一点，您不应该使用任何循环，而应该使用 numpy 矢量化操作来完成所有计算（为了在此设定期望值，我们应该注意到，例如，我们的参考解决方案只包含一行代码）。

需要注意的是，对于 softmax loss 的 "真正 "实现，你会希望缩放对数以防止数值溢出，但我们在这里不会担心这个问题（即使你不担心这个问题，赋值的其余部分也能正常工作）。 下面的代码将运行测试用例。

In [ ]:
!python3 -m pytest -k "softmax_loss"

Then run the submission.

然后运行提交程序。

In [ ]:
!python3 -m mugrade submit YOUR_GRADER_KEY_HERE hw0 -k "softmax_loss"

## Question 4: Stochastic gradient descent for softmax regression

In this question you will implement stochastic gradient descent (SGD) for (linear) softmax regression.  In other words, as discussed in lecture on 9/1, we will consider a hypothesis function that makes $n$-dimensional inputs to $k$-dimensional logits via the function
\begin{equation}
h(x) = \Theta^T x
\end{equation}
where $x \in \mathbb{R}^n$ is the input, and $\Theta \in \mathbb{R}^{n \times k}$ are the model parameters.  Given a dataset $\{(x^{(i)} \in \mathbb{R}^n, y^{(i)} \in \{1,\ldots,k\})\}$, for $i=1,\ldots,m$, the optimization problem associated with softmax regression is thus given by
\begin{equation}
\DeclareMathOperator*{\minimize}{minimize}
\minimize_{\Theta} \; \frac{1}{m} \sum_{i=1}^m \ell_{\mathrm{softmax}}(\Theta^T x^{(i)}, y^{(i)}).
\end{equation}

Recall from class that the gradient of the linear softmax objective is given by
\begin{equation}
\nabla_\Theta \ell_{\mathrm{softmax}}(\Theta^T x, y) = x (z - e_y)^T
\end{equation}
where
\begin{equation}
\DeclareMathOperator*{\normalize}{normalize}
z = \frac{\exp(\Theta^T x)}{1^T \exp(\Theta^T x)} \equiv \normalize(\exp(\Theta^T x))
\end{equation}
(i.e., $z$ is just the normalized softmax probabilities), and where $e_y$ denotes the $y$th unit basis, i.e., a vector of all zeros with a one in the $y$th position.

We can also write this in the more compact notation we discussed in class.  Namely, if we let $X \in \mathbb{R}^{m \times n}$ denote a design matrix of some $m$ inputs (either the entire dataset or a minibatch), $y \in \{1,\ldots,k\}^m$ a corresponding vector of labels, and overloading $\ell_{\mathrm{softmax}}$ to refer to the average softmax loss, then
\begin{equation}
\nabla_\Theta \ell_{\mathrm{softmax}}(X \Theta, y) = \frac{1}{m} X^T (Z - I_y)
\end{equation}
where
\begin{equation}
Z = \normalize(\exp(X \Theta)) \quad \mbox{(normalization applied row-wise)}
\end{equation}
denotes the matrix of logits, and $I_y \in \mathbb{R}^{m \times k}$ represents a concatenation of one-hot bases for the labels in $y$.

Using these gradients, implement the `softmax_regression_epoch()` function, which runs a single epoch of SGD (one pass over a data set) using the specified learning rate / step size `lr` and minibatch size `batch`.  As described in the docstring, your function should modify the `Theta` array in-place.  After implementation, run the tests.

## 问题 4：软最大回归的随机梯度下降法

在本题中，你将实现（线性）软最大回归的随机梯度下降（SGD）。 换句话说，正如 9/1 号讲座所讨论的，我们将考虑一个假设函数，该函数通过以下函数使 $n$ 维输入变为 $k$ 维对数
\begin{equation}
h(x) = \Theta^T x
\end{equation}
where $x \in \mathbb{R}^n$ is the input, and $\Theta \in \mathbb{R}^{n \times k}$ are the model parameters.  Given a dataset $\{(x^{(i)} \in \mathbb{R}^n, y^{(i)} \in \{1,\ldots,k\})\}$, for $i=1,\ldots,m$, the optimization problem associated with softmax regression is thus given by
\begin{equation}
\DeclareMathOperator*{\minimize}{minimize}
\minimize_{\Theta} \; \frac{1}{m} \sum_{i=1}^m \ell_{\mathrm{softmax}}(\Theta^T x^{(i)}, y^{(i)}).
\end{equation}
回想一下，线性软最大目标的梯度为
\begin{equation}
\nabla_\Theta \ell_{\mathrm{softmax}}(\Theta^T x, y) = x (z - e_y)^T
\end{equation}
where
\begin{equation}
\DeclareMathOperator*{\normalize}{normalize}
z = \frac{\exp(\Theta^T x)}{1^T \exp(\Theta^T x)} \equiv \normalize(\exp(\Theta^T x))
\end{equation}
(即 $z$ 只是归一化的软最大概率(),)其中 $e_y$ 表示第 y$ 个单位基，即在第 y$ 个位置为 1 的所有零的向量。

我们也可以用课堂上讨论过的更简洁的符号来写。 也就是说，如果我们让 $X \ in \mathbb{R}^{m \times n}$ 表示某个 $m$ 输入的设计矩阵（可以是整个数据集，也可以是一个小批量输入），$y \ in \{1,\ldots,k\}^m$ 是一个相应的标签向量，并重载 $ell_{\mathrm{softmax}}$ 来表示平均软最大损失，则
\begin{equation}
\nabla_\Theta \ell_{\mathrm{softmax}}(X \Theta, y) = \frac{1}{m} X^T (Z - I_y)
\end{equation}
where
\begin{equation}
Z = \normalize(\exp(X \Theta)) \quad \mbox{(normalization applied row-wise)}
\end{equation}
表示对数矩阵，而 $I_y \in \mathbb{R}^{m \times k}$ 表示 $y$ 中标签的单次基数。

使用这些梯度，实现 `softmax_regression_epoch()` 函数，该函数使用指定的学习率/步长 `lr` 和最小批量 `batch` 运行 SGD 的单个 epoch（对数据集的一次传递）。 正如文档中所述，你的函数应就地修改 `Theta` 数组。 执行后，运行测试。

In [ ]:
!python3 -m pytest -k "softmax_regression_epoch and not cpp"

And then run the submission.

然后运行提交程序。

In [ ]:
!python3 -m mugrade submit YOUR_GRADER_KEY_HERE hw0 -k "softmax_regression_epoch and not cpp"

### Training MNIST with softmax regression

Although it's not a part of the tests, now that you have written this code, you can also try training a full MNIST linear classifier using SGD.  For this you can use the `train_softmax()` function in the `src/simple_ml.py` file (we have already written this function for you, so you don't need to write it yourself, though you can take a look to see what it's doing).  

You can see how this works using the following code.  For reference, as seen below, our implementation runs in ~3 seconds on Colab, and achieves 7.97% error.

### 使用 softmax 回归训练 MNIST

虽然这不是测试的一部分，但既然你已经编写了这段代码，也可以尝试使用 SGD 训练一个完整的 MNIST 线性分类器。 为此，您可以使用 `src/simple_ml.py` 文件中的 `train_softmax()` 函数（我们已经为您编写了这个函数，因此您无需自己编写，不过您可以看一看它在做什么）。

您可以使用下面的代码了解其工作原理。 作为参考，如下所示，我们的实现在 Colab 上的运行时间约为 3 秒，误差率为 7.97%。

In [26]:
import sys
sys.path.append("src/")
from simple_ml import train_softmax, parse_mnist

X_tr, y_tr = parse_mnist("data/train-images-idx3-ubyte.gz", 
                         "data/train-labels-idx1-ubyte.gz")
X_te, y_te = parse_mnist("data/t10k-images-idx3-ubyte.gz",
                         "data/t10k-labels-idx1-ubyte.gz")

train_softmax(X_tr, y_tr, X_te, y_te, epochs=10, lr=0.2, batch=100)

| Epoch | Train Loss | Train Err | Test Loss | Test Err |
|     0 |    0.35134 |   0.10182 |   0.33588 |  0.09400 |
|     1 |    0.32142 |   0.09268 |   0.31086 |  0.08730 |
|     2 |    0.30802 |   0.08795 |   0.30097 |  0.08550 |
|     3 |    0.29987 |   0.08532 |   0.29558 |  0.08370 |
|     4 |    0.29415 |   0.08323 |   0.29215 |  0.08230 |
|     5 |    0.28981 |   0.08182 |   0.28973 |  0.08090 |
|     6 |    0.28633 |   0.08085 |   0.28793 |  0.08080 |
|     7 |    0.28345 |   0.07997 |   0.28651 |  0.08040 |
|     8 |    0.28100 |   0.07923 |   0.28537 |  0.08010 |
|     9 |    0.27887 |   0.07847 |   0.28442 |  0.07970 |


## Question 5: SGD for a two-layer neural network

Now that you've written SGD for a linear classifier, let's consider the case of a simple two-layer neural network.  Specifically, for input $x \in \mathbb{R}^n$, we'll consider a two-layer neural network (without bias terms) of the form
\begin{equation}
z = W_2^T \mathrm{ReLU}(W_1^T x)
\end{equation}
where $W_1 \in \mathbb{R}^{n \times d}$ and $W_2 \in \mathbb{R}^{d \times k}$ represent the weights of the network (which has a $d$-dimensional hidden unit), and where $z \in \mathbb{R}^k$ represents the logits output by the network.  We again use the softmax / cross-entropy loss, meaning that we want to solve the optimization problem
\begin{equation}
\minimize_{W_1, W_2} \;\; \frac{1}{m} \sum_{i=1}^m \ell_{\mathrm{softmax}}(W_2^T \mathrm{ReLU}(W_1^T x^{(i)}), y^{(i)}).
\end{equation}
Or alternatively, overloading the notation to describe the batch form with matrix $X \in \mathbb{R}^{m \times n}$, this can also be written 
\begin{equation}
\minimize_{W_1, W_2} \;\; \ell_{\mathrm{softmax}}(\mathrm{ReLU}(X W_1) W_2, y).
\end{equation}

Using the chain rule, we can derive the backpropagation updates for this network (we'll briefly cover these in class, on 9/8, but also provide the final form here for ease of implementation).  Specifically, let
\begin{equation}
\begin{split}
Z_1 \in \mathbb{R}^{m \times d} & = \mathrm{ReLU}(X W_1) \\
G_2 \in \mathbb{R}^{m \times k} & = \normalize(\exp(Z_1 W_2)) - I_y \\
G_1 \in \mathbb{R}^{m \times d} & = \mathrm{1}\{Z_1 > 0\} \circ (G_2 W_2^T)
\end{split}
\end{equation}
where $\mathrm{1}\{Z_1 > 0\}$ is a binary matrix with entries equal to zero or one depending on whether each term in $Z_1$ is strictly positive and where $\circ$ denotes elementwise multiplication.  Then the gradients of the objective are given by
\begin{equation}
\begin{split}
\nabla_{W_1} \ell_{\mathrm{softmax}}(\mathrm{ReLU}(X W_1) W_2, y) & = \frac{1}{m} X^T G_1  \\
\nabla_{W_2} \ell_{\mathrm{softmax}}(\mathrm{ReLU}(X W_1) W_2, y) & = \frac{1}{m} Z_1^T G_2.  \\
\end{split}
\end{equation}

**Note:** If the details of these precise equations seem a bit cryptic to you (prior to the 9/8 lecture), don't worry too much.  These _are_ just the standard backpropagation equations for a two-layer ReLU network: the $Z_1$ term just computes the "forward" pass while the $G_2$ and $G_1$ terms denote the backward pass.  But the precise form of the updates can vary depending upon the notation you've used for neural networks, the precise ways you formulate the losses, if you've derived these previously in matrix form, etc.  If the notation seems like it might be familiar from when you've seen deep networks in the past, and makes more sense after the 9/8 lecture, that is more than sufficient in terms of background (after all, the whole _point_ of deep learning systems, to some extent, is that we don't need to bother with these manual calculations).  But if these entire concepts are _completely_ foreign to you, then it may be better to take a separate course on ML and neural networks prior to this course, or at least be aware that there will be substantial catch-up work to do for the course.

Using these gradients, now write the `nn_epoch()` function in the `src/simple_ml.py` file.  As with the previous question, your solution should modify the `W1` and `W2` arrays in place.  After implementing the function, run the following test.  Be sure to use matrix operations as indicated by the expresssions above to implement the function: this will be _much_ faster, and more efficient, than attempting to use loops (and it requires far less code).

## 问题5：双层神经网络随机梯度下降

在完成线性分类器的SGD实现后，我们现在考虑一个简单的双层神经网络。具体而言，对于输入$x \in \mathbb{R}^n$，我们考察如下形式的双层神经网络（无偏置项）：
\begin{equation}
z = W_2^T \mathrm{ReLU}(W_1^T x)
\end{equation}
其中$W_1 \in \mathbb{R}^{n \times d}$和$W_2 \in \mathbb{R}^{d \times k}$表示网络权重（包含$d$维隐藏单元），$z \in \mathbb{R}^k$表示网络输出的logits。我们继续使用softmax/交叉熵损失函数，即需要求解优化问题：
\begin{equation}
\minimize_{W_1, W_2} \;\; \frac{1}{m} \sum_{i=1}^m \ell_{\mathrm{softmax}}(W_2^T \mathrm{ReLU}(W_1^T x^{(i)}), y^{(i)}).
\end{equation}
或者通过重载矩阵$X \in \mathbb{R}^{m \times n}$表示批量形式的等价表达式：
\begin{equation}
\minimize_{W_1, W_2} \;\; \ell_{\mathrm{softmax}}(\mathrm{ReLU}(X W_1) W_2, y).
\end{equation}

运用链式法则可以推导该网络的反向传播更新规则（我们将在9月8日课堂简要讲解，此处直接给出最终形式以便实现）。具体地，令：
\begin{equation}
\begin{split}
Z_1 \in \mathbb{R}^{m \times d} & = \mathrm{ReLU}(X W_1) \\
G_2 \in \mathbb{R}^{m \times k} & = \normalize(\exp(Z_1 W_2)) - I_y \\
G_1 \in \mathbb{R}^{m \times d} & = \mathrm{1}\{Z_1 > 0\} \circ (G_2 W_2^T)
\end{split}
\end{equation}
其中$\mathrm{1}\{Z_1 > 0\}$是根据$Z_1$各元素是否严格大于零生成的二元矩阵，$\circ$表示逐元素乘法。则目标函数的梯度为：
\begin{equation}
\begin{split}
\nabla_{W_1} \ell_{\mathrm{softmax}}(\mathrm{ReLU}(X W_1) W_2, y) & = \frac{1}{m} X^T G_1  \\
\nabla_{W_2} \ell_{\mathrm{softmax}}(\mathrm{ReLU}(X W_1) W_2, y) & = \frac{1}{m} Z_1^T G_2.  \\
\end{split}
\end{equation}

**说明：** 如果这些方程的具体细节在9月8日课前看起来晦涩难懂，请不必过分担忧。这些只是标准的双层ReLU网络反向传播方程——$Z_1$项执行前向传播，而$G_2$和$G_1$项对应反向传播。方程的具体形式可能因神经网络记号、损失函数构建方式或矩阵推导过程而有所差异。只要这些记号与你之前接触的深度网络内容类似，或在9月8日课后能够理解，就完全具备所需基础（毕竟深度学习系统的核心价值就在于我们不必手动进行这些计算）。但如果这些概念对你而言完全陌生，建议先修读机器学习与神经网络相关课程，或做好充分的课外补习准备。

根据上述梯度计算公式，请在`src/simple_ml.py`文件中实现`nn_epoch()`函数。与前一问题相同，你的解决方案应直接修改`W1`和`W2`数组。实现后运行测试代码。务必按照上述表达式使用矩阵运算来实现函数——这将比循环实现更高效快速，且代码更为简洁。

In [ ]:
!python3 -m pytest -k "nn_epoch"

And finally submit for autograding.

最后提交自动交易。

In [ ]:
!python3 -m mugrade submit YOUR_GRADER_KEY_HERE hw0 -k "nn_epoch"

### Training a full neural network

As before, though it isn't a strict necessity to pass the autograder, it's rather fun to see how well you can use your neural network function to train an MNIST classifier.  Analogous to the softmax regression case, there is a `train_nn()` function in the `simple_ml.py` file you can use to train this two-layer network via SGD with multiple epochs.  Here is code, for example, that trains a two-layer network with 400 hidden units.

### 训练完整的神经网络

与之前一样，虽然通过 autograder 并不是一个严格的必要条件，但看看使用神经网络函数训练 MNIST 分类器的效果如何，还是相当有趣的。 与软最大回归的情况类似，在 `simple_ml.py` 文件中有一个 `train_nn()` 函数，你可以用它来通过 SGD 训练这个具有多个 epoch 的双层网络。 例如，下面的代码可以训练一个具有 400 个隐藏单元的双层网络。

In [25]:
import sys

# Reload the simple_ml module which has been cached from the earlier experiment
import importlib
import simple_ml
importlib.reload(simple_ml)

sys.path.append("src/")
from simple_ml import train_nn, parse_mnist

X_tr, y_tr = parse_mnist("data/train-images-idx3-ubyte.gz", 
                         "data/train-labels-idx1-ubyte.gz")
X_te, y_te = parse_mnist("data/t10k-images-idx3-ubyte.gz",
                         "data/t10k-labels-idx1-ubyte.gz")
train_nn(X_tr, y_tr, X_te, y_te, hidden_dim=400, epochs=20, lr=0.2)

| Epoch | Train Loss | Train Err | Test Loss | Test Err |
|     0 |    0.15324 |   0.04697 |   0.16305 |  0.04920 |
|     1 |    0.09854 |   0.02923 |   0.11604 |  0.03660 |
|     2 |    0.07392 |   0.02163 |   0.09750 |  0.03200 |
|     3 |    0.06006 |   0.01757 |   0.08825 |  0.02960 |
|     4 |    0.04869 |   0.01368 |   0.08147 |  0.02620 |
|     5 |    0.04061 |   0.01093 |   0.07698 |  0.02380 |
|     6 |    0.03494 |   0.00915 |   0.07446 |  0.02320 |
|     7 |    0.03027 |   0.00758 |   0.07274 |  0.02320 |
|     8 |    0.02674 |   0.00650 |   0.07103 |  0.02240 |
|     9 |    0.02373 |   0.00552 |   0.06989 |  0.02150 |
|    10 |    0.02092 |   0.00477 |   0.06870 |  0.02130 |
|    11 |    0.01914 |   0.00403 |   0.06837 |  0.02130 |
|    12 |    0.01705 |   0.00325 |   0.06748 |  0.02150 |
|    13 |    0.01541 |   0.00272 |   0.06688 |  0.02130 |
|    14 |    0.01417 |   0.00232 |   0.06657 |  0.02090 |
|    15 |    0.01282 |   0.00195 |   0.06591 |  0.02040 |
|    16 |    0

This takes about 30 seconds to run on Colab for our implementation, and as seen above, it achieve an error of 1.89\% on MNIST.  Not bad for less than 20 lines of code or so...

在 Colab 上运行我们的实现大约需要 30 秒，如上图所示，它在 MNIST 上的误差为 1.89/%。 对于不到 20 行的代码来说，这已经很不错了...

## Question 6: Softmax regression in C++

The final question in this homework requires you to implement the same function you did in Question 4, a function that runs a single epoch of linear softmax regression.  But here, you will do so with a C++ implementation, rather than a Python one. (Strictly speaking, the actual implementation here is more like raw C, but we use C++ features to build the interface to Python using the [pybind11](https://pybind11.readthedocs.io) library, which you will also use to interface between C++ and Python in later homeworks.  Although there are other alternatives, pybind11 library is relatively nice as an interface, as it is a header-only library, and allows you to implement the entire Python/C++ interface within a single C++ source library.)

The C++ file you'll implement things in is the `src/simple_ml_ext.cpp` file.  Let's take a look at the relevant portion of the file.  You will specifically implement your code in the following function

```cpp
void softmax_regression_epoch_cpp(const float *X, const unsigned char *y, 
								  float *theta, size_t m, size_t n, size_t k, 
								  float lr, size_t batch)
{
    /**
     * A C++ version of the softmax regression epoch code.  This should run a 
     * single epoch over the data defined by X and y (and sizes m,n,k), and
     * modify theta in place.  Your function will probably want to allocate
     * (and then delete) some helper arrays to store the logits and gradients.
     * 
     * Args:
     *     X (const float *): pointer to X data, of size m*n, stored in row 
     *          major (C) format
     *     y (const unsigned char *): pointer to y data, of size m
     *     theta (float *): pointer to theta data, of size n*k, stored in row
     *          major (C) format
     *     m (size_t): number of examples
     *     n (size_t): input dimension
     *     k (size_t): number of classes
     *     lr (float): learning rate / SGD step size
     *     batch (int): SGD minibatch size
     * 
     * Returns:
     *     (None)
     */

    /// YOUR CODE HERE
    
    /// END YOUR CODE
}
```
Let's unpack the arguments to this function a bit.  The function essentially mirrors that of the Python implementation, but requires passing some additional arguments because we are operating on raw pointers to the array data rather than any sort of higher-level "matrix" data structure.  Specifically, `X`, `y`, and `theta` are pointers to the raw data of the corresponding numpy arrays from the previous section; for 2D arrays, these are stored in C-style (row-major) format, meaning that the first row of $X$ is stored sequentially as the first bytes in `X`, then the second row, etc (this contrasts with _column major_ ordering, which stores the first _column_ of the matrirx sequentially, then the second column, etc).  We also assuming there is no padding in the data; that is, the second row begins immediately after the first row, with no additional bytes added, e.g., to align the memory to a certain boundary (all these issues will be mentioned in subsequent discussion in the course, but avoided for now). Of course, because only the raw data is passed into the function, in order to know the actual sizes of the underlying matrices, we also need to pass these sizes explicitly to the function, which is what is provided by the `m`, `n`, and `k` arguments.

As an illustration of how to access the data, note that because `X` represents a row-major, $m \times n$ matrix, if we want to access the $(i,j)$ element of $X$ (the element in the $i$th row and the $j$th column), we would use the index
```cpp
X[i*n + j]
```
i.e., because `X` is $n$ columns, and stores it's columns sequentially, we'd need to access the `X[i*n]` element to access the $i$th row; and if we want to access the $j$th element in this row, we have the expression above.  The same logic would apply to the `theta` matrix, but importantly, because `theta` is a $n \times k$ matrix, to access it's $(i,j)$ element you would use the index
```cpp
theta[i*k + j]
```
Unlike in Python, you need to be very careful when accessing memory directly like this in C++, and get very used to this kind of notation (or build additional data structures that help you access things in a more intuitive fashion, but for this assignment you should just stick to the raw indexing).


The second piece of importance for the implementation is the pybind11 code that actually provides the Python interface
```cpp
PYBIND11_MODULE(simple_ml_ext, m) {
    m.def("softmax_regression_epoch_cpp", 
    	[](py::array_t<float, py::array::c_style> X, 
           py::array_t<unsigned char, py::array::c_style> y, 
           py::array_t<float, py::array::c_style> theta,
           float lr,
           int batch) {
        softmax_regression_epoch_cpp(
        	static_cast<const float*>(X.request().ptr),
            static_cast<const unsigned char*>(y.request().ptr),
            static_cast<float*>(theta.request().ptr),
            X.request().shape[0],
            X.request().shape[1],
            theta.request().shape[1],
            lr,
            batch
           );
    },
    py::arg("X"), py::arg("y"), py::arg("theta"), 
    py::arg("lr"), py::arg("batch"));
}
```
This code is provided for you in the file, and you should not change it at all.  But for those who are curious, this code essentially just extracts the raw pointers from the provided inputs (using pybinds numpy interface), and then calls the corresponding `softmax_regression_epoch_cpp` function.

Using all this as background, implement the `softmax_regression_epoch_cpp` to accomplish the same updates as your Python implementation did.  Note that because you are just accessing the raw data, you will need to perform all the matrix-vector products manually, rather that rely on numpy to do all the matrix operations for you (**note: do not use an external matrix library like Eigen for this assignment, but code the multiplication yourself ... it is a relatively simple one**). After you do so, you can test the implementation using the following commands.

## 问题6：C++实现Softmax回归

本作业的最后一项要求是实现与问题4相同的功能——运行单轮线性softmax回归。但这里需要使用C++而非Python实现。（严格来说这更像是原始C语言实现，但我们使用C++特性通过[pybind11](https://pybind11.readthedocs.io)库构建Python接口，后续作业中也将继续使用该库进行C++/Python交互。虽然存在其他方案，但pybind11作为仅头文件库，能在一个C++源文件中实现完整的Python/C++接口，因此是较优选择。）

实现代码将位于`src/simple_ml_ext.cpp`文件中。重点实现以下函数：

```cpp
void softmax_regression_epoch_cpp(const float *X, const unsigned char *y, 
								  float *theta, size_t m, size_t n, size_t k, 
								  float lr, size_t batch)
{
    /**
     * Softmax回归的C++单轮训练实现。该函数应对X和y定义的数据（大小为m,n,k）运行单个训练轮次，
     * 并原地修改theta参数。函数内部需要分配（随后释放）辅助数组来存储logits和梯度。
     * 
     * 参数说明：
     *     X (const float *)：指向m*n维X数据的指针，按行主序（C格式）存储
     *     y (const unsigned char *)：指向m维y数据的指针
     *     theta (float *)：指向n*k维theta参数的指针，按行主序存储
     *     m (size_t)：样本数量
     *     n (size_t)：输入维度
     *     k (size_t)：类别数量
     *     lr (float)：学习率/SGD步长
     *     batch (int)：SGD小批量大小
     */
}
```

需要特别注意参数的内存布局：所有二维数组都采用C风格（行主序）存储，即矩阵的首行数据连续存储，次行紧随其后（与列主序存储相反）。例如访问X矩阵的第(i,j)个元素（第i行第j列）应使用：

```cpp
X[i*n + j]  // 因每行有n列元素，i*n定位到行首
```

对于theta矩阵（n×k维），其(i,j)元素的访问方式则为：

```cpp
theta[i*k + j]  // 注意theta每行有k个元素
```

文件中的pybind11接口代码已提供完备，无需修改：

```cpp
PYBIND11_MODULE(simple_ml_ext, m) {
    m.def("softmax_regression_epoch_cpp", [...]); // 完整接口定义
}
```

该接口代码负责从numpy数组提取原始指针，并调用对应的C++函数。实现要求：
1. 保持与Python版本相同的算法逻辑
2. 手动实现所有矩阵运算（禁止使用Eigen等外部矩阵库）
3. 通过分配临时数组存储logits和梯度
4. 注意内存访问的安全性

完成实现后，可使用提供的测试代码验证正确性。

In [28]:
!make
!python3 -m pytest -k "softmax_regression_epoch_cpp"

c++ -O3 -Wall -shared -std=c++11 -fPIC -undefined dynamic_lookup $(python3 -m pybind11 --includes) src/simple_ml_ext.cpp -o src/simple_ml_ext.so
============================= test session starts ==============================
platform darwin -- Python 3.7.3, pytest-4.3.1, py-1.8.0, pluggy-0.9.0
rootdir: /Users/zkolter/Dropbox/class/10-714/homework/hw0, inifile:
plugins: remotedata-0.3.1, openfiles-0.3.2, doctestplus-0.3.0, arraydiff-0.3
collected 6 items / 5 deselected / 1 selected                                  

tests/test_simple_ml.py .                                                [100%]

==================== 1 passed, 5 deselected in 0.49 seconds ====================


Note that unlike our code before, we had to actually compile the C++ extension in order to run an test it.  This will be needed whenever there are C++ components of your code, but in all such cases we will include Makefiles that should compile all the relevant functions and include the necessary libraries/headers.  And finally, let's also submit the results to mugrade.

请注意，与之前的代码不同，我们必须实际编译 C++ 扩展才能运行和测试它。 只要代码中有 C++ 组件，就需要这样做，但在所有这些情况下，我们都会包含 Makefile 文件，这些文件应能编译所有相关函数并包含必要的库/头文件。 最后，我们还要将结果提交给 mugrade。

In [ ]:
!python3 -m mugrade submit YOUR_GRADER_KEY_HERE hw0 -k "softmax_regression_epoch_cpp"

### Training a full softmax regression classifier with the C++ version

Let's finally try training the whole softmax regression classifier using our "direct memory acesss" C++ version.  If the previous Python version took ~3 seconds, this should be blazing fast, right?

### 使用 C++ 版本训练完整的 softmax 回归分类器

最后，让我们尝试使用 "直接内存 "的 C++ 版本训练整个 softmax 回归分类器。 如果之前的 Python 版本需要 ~3 秒，那么这个版本应该很快，对吗？

In [30]:
import sys
sys.path.append("src/")

# Reload the simple_ml module to include the newly-compiled C++ extension
import importlib
import simple_ml
importlib.reload(simple_ml)

from simple_ml import train_softmax, parse_mnist

X_tr, y_tr = parse_mnist("data/train-images-idx3-ubyte.gz", 
                         "data/train-labels-idx1-ubyte.gz")
X_te, y_te = parse_mnist("data/t10k-images-idx3-ubyte.gz",
                         "data/t10k-labels-idx1-ubyte.gz")

train_softmax(X_tr, y_tr, X_te, y_te, epochs=10, lr = 0.2, batch=100, cpp=True)

| Epoch | Train Loss | Train Err | Test Loss | Test Err |
|     0 |    0.35134 |   0.10182 |   0.33588 |  0.09400 |
|     1 |    0.32142 |   0.09268 |   0.31086 |  0.08730 |
|     2 |    0.30802 |   0.08795 |   0.30097 |  0.08550 |
|     3 |    0.29987 |   0.08532 |   0.29558 |  0.08370 |
|     4 |    0.29415 |   0.08323 |   0.29215 |  0.08230 |
|     5 |    0.28981 |   0.08182 |   0.28973 |  0.08090 |
|     6 |    0.28633 |   0.08085 |   0.28793 |  0.08080 |
|     7 |    0.28345 |   0.07997 |   0.28651 |  0.08040 |
|     8 |    0.28100 |   0.07923 |   0.28537 |  0.08010 |
|     9 |    0.27887 |   0.07847 |   0.28442 |  0.07970 |


As expected, the numbers match exactly our Python version, and the code is ... about 5 times slower?!  What is going on here?  Well, it turns out that the "manual" matrix multiplication code you probably wrote for the C++ version is extremely inefficient.  While Python itself is a slow, interpreted language, numpy itself is backed by matrix multiplications written in C (or, more likely, Fortran, believe it or not), that have been highly optmized to make use of vector operations, the cache hierarchy of different processors, and other features that are essential for efficient numerical operations.  We will cover these details much more in later lectures, and you'll even write a matrix library that can actually perform these operations relatively efficiently (at least for some special cases ... it's honestly not that easy to beat numpy in general).

But for now, assuming your code recreates the Python behavior, you're done with the assignment, and can get ready for our next dive into automatic differentiation.

不出所料，这些数字与我们的 Python 版本完全一致，而代码却......慢了大约 5 倍！ 这到底是怎么回事？ 事实证明，你可能为 C++ 版本编写的 "手动 "矩阵乘法代码效率极低。 Python 本身是一种慢速的解释型语言，而 numpy 本身则是由 C 语言（或者更有可能是 Fortran，信不信由你）编写的矩阵乘法代码支持的，这些矩阵乘法代码已经被高度优化，以利用矢量操作、不同处理器的缓存层次结构以及对高效数值运算至关重要的其他特性。 我们将在以后的讲座中详细介绍这些细节，你甚至可以编写一个能相对高效地执行这些操作的矩阵库（至少在某些特殊情况下是这样......老实说，要在一般情况下打败 numpy 并不容易）。

但现在，假设你的代码重现了 Python 的行为，你就完成了作业，可以准备下一次深入学习自动微分了。